In [1]:
import os
import pandas as pd
import numpy as np
import re

#Для работы с Excel
import xlrd, xlwt
from xlsxwriter.utility import xl_rowcol_to_cell
import openpyxl

pd.set_option('display.max_columns', 500)
pd.options.display.max_colwidth=250

Для реализации задачи, обработаем Справочник предметов государственного заказа города Москвы в файле: Приложение 4.Список СПГЗ 27_5_2022.xlsx

In [2]:
# Загрузим его
url = 'https://docs.google.com/spreadsheets/d/1S-xYf3FLnptFvMMvCcVc2Doe1xYI8KMEMB2aE4yvTmU/edit?usp=sharing'
url.split('/')
id = url.split('/')[5]
spgz_list = pd.read_excel(f'https://docs.google.com/spreadsheets/d/{id}/export?format=xlsx')

spgz_list.head(3)

,ID,Дата и время создания,Дата последнего изменения,КПГЗ,Наименование СПГЗ,Описание,Единицы измерения,ОКПД,ОКПД 2,Код номенклатуры МЭР,Код КТРУ,"Информация о полезных моделях, промышленных образцах",Признак стандартизированной позиции,Признак возможности определения поставщика путем проведения конкурса с ограниченным участием,Признак применимости критерия жизненного цикла,Признак результата интеллектуальной деятельности либо НИОКР,Признак создания произведения искусства,Признак возможности определения поставщика путем проведения двухэтапного конкурса,Преподавательские услуги физическим лицом,Услуги экскурсоводов физическим лицом,Признак возможности проведения закупки на единицу продукции,Требует нормирования,Проект,Есть ПЦП,Наименование шаблона ТЗ
0,40209538,15.08.2019 12:31:00,19.02.2020 00:00:00,03.30.06.04 МЕТРОЛОГИЧЕСКОЕ ОБСЛУЖИВАНИЕ НЕ МЕДИЦИНСКИХ ИЗДЕЛИЙ,Поверка грузов к грузопоршневому манометру,Поверка грузов к грузопоршневому манометру,Штука,NaN,71.12.40.129,NaN,NaN,NaN,Нет,Нет,Нет,Нет,Нет,Нет,Нет,Нет,Да,Нет,Нет,Нет,NaN
1,40209531,10.08.2019 23:19:00,19.02.2020 00:00:00,03.30.06.04 МЕТРОЛОГИЧЕСКОЕ ОБСЛУЖИВАНИЕ НЕ МЕДИЦИНСКИХ ИЗДЕЛИЙ,Поверка грузопоршневого манометра,Поверка грузопоршневого манометра,Штука,NaN,71.12.40.129,NaN,NaN,NaN,Нет,Нет,Нет,Нет,Нет,Нет,Нет,Нет,Да,Нет,Нет,Нет,NaN
2,40209536,15.08.2019 12:23:01,19.02.2020 00:00:00,03.30.06.04 МЕТРОЛОГИЧЕСКОЕ ОБСЛУЖИВАНИЕ НЕ МЕДИЦИНСКИХ ИЗДЕЛИЙ,Поверка измерителя параметров электроизоляции,Поверка измерителя параметров электроизоляции,Штука,NaN,71.12.40.129,NaN,NaN,NaN,Нет,Нет,Нет,Нет,Нет,Нет,Нет,Нет,Да,Нет,Нет,Нет,NaN


In [3]:
spgz_list = (
    spgz_list[['ID', 'КПГЗ', 'Наименование СПГЗ', 'Единицы измерения', 'ОКПД', 'ОКПД 2']]
    .rename(columns={'ID': 'id',
                    'КПГЗ': 'kpgz',
                    'Наименование СПГЗ': 'spgz',
                    'Единицы измерения': 'count',
                    'ОКПД': 'okpd',
                    'ОКПД 2': 'okpd_2'})
)
spgz_list.head()

,id,kpgz,spgz,count,okpd,okpd_2
0,40209538,03.30.06.04 МЕТРОЛОГИЧЕСКОЕ ОБСЛУЖИВАНИЕ НЕ МЕДИЦИНСКИХ ИЗДЕЛИЙ,Поверка грузов к грузопоршневому манометру,Штука,NaN,71.12.40.129
1,40209531,03.30.06.04 МЕТРОЛОГИЧЕСКОЕ ОБСЛУЖИВАНИЕ НЕ МЕДИЦИНСКИХ ИЗДЕЛИЙ,Поверка грузопоршневого манометра,Штука,NaN,71.12.40.129
2,40209536,03.30.06.04 МЕТРОЛОГИЧЕСКОЕ ОБСЛУЖИВАНИЕ НЕ МЕДИЦИНСКИХ ИЗДЕЛИЙ,Поверка измерителя параметров электроизоляции,Штука,NaN,71.12.40.129
3,40209532,03.30.06.04 МЕТРОЛОГИЧЕСКОЕ ОБСЛУЖИВАНИЕ НЕ МЕДИЦИНСКИХ ИЗДЕЛИЙ,Поверка измерителя сопротивления заземления,Штука,NaN,71.12.40.129
4,40209559,03.30.06.04 МЕТРОЛОГИЧЕСКОЕ ОБСЛУЖИВАНИЕ НЕ МЕДИЦИНСКИХ ИЗДЕЛИЙ,Поверка индукционного трансформатора тока,Штука,NaN,71.12.40.129


In [4]:
# Сохраним очищенный справочник
spgz_list.to_excel('list_spgz_27_5_2022.xlsx', index=False)